In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')
df19 = pd.read_csv('../data/feature_color_checkout_ult15dias.csv', low_memory=False, index_col='person')

labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
# Al que tenga "diferencia_de_count_de_eventos.csv" hay que rellenarlo
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17, how='inner')
df_unidos = df_unidos.join(df18, how='inner')
df_unidos = df_unidos.join(df19, how='inner')

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 101)

In [3]:
features = df_test.columns.tolist()
features.remove('label')

In [4]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [14]:
param_grid = { 
    'num_leaves': [1000, 500, 100],
    'max_depth': [3,5, 7, 10],
    'reg_lambda': [0.1, 1, 10]
}

In [15]:
_lgb = lgb.LGBMClassifier(random_state=80, class_weight='balanced', colsample_bytree=0.9)
CV_lgb = GridSearchCV(estimator=_lgb, param_grid=param_grid, cv=10, scoring='roc_auc', n_jobs=4)
CV_lgb.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
        colsample_bytree=0.9, importance_type='split', learning_rate=0.1,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective=None, random_state=80, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'num_leaves': [1000, 500, 100], 'max_depth': [3, 5, 7, 10], 'reg_lambda': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [16]:
CV_lgb.best_params_

{'max_depth': 3, 'num_leaves': 1000, 'reg_lambda': 10}

In [17]:
CV_lgb.best_score_

0.8731341905903864

In [18]:
#model_lgb = lgb.LGBMClassifier(random_state=80,\
#                               n_estimators=1000, learning_rate=0.01, max_depth=4, gamma=1)
#model_lgb.fit(X_train, y_train)
#y_pred = model_lgb.predict_proba(X_test)
y_pred = CV_lgb.predict_proba(X_test)
y_pred_proba = [p[1] for p in y_pred]
print(roc_auc_score(y_test, y_pred_proba))

0.8530402930402929


In [19]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos[features], how='inner')

In [20]:
df_events.shape

(19415, 100)

In [21]:
kaggle_pred = CV_lgb.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [22]:
df_submit.to_csv('../submit_lgbm_v5.csv')